Простой чат-агент без инструментов:

In [ ]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI

load_dotenv()

True

In [ ]:
# создаем коннектор
llm = ChatOpenAI(
    model=os.environ.get("BASE_MODEL"),
    base_url=os.environ.get("BASE_URL"),
    api_key=os.environ.get("LLM_API_KEY"),
    temperature=0.2,
)

def chat(user_input: str):
    response = llm.invoke(user_input)
    return response.content

# проверим, что все работает
answer = chat("What is Python?")
print(answer)


**Python** is a high-level, interpreted, general-purpose programming language known for its simplicity, readability, and versatility. Created by **Guido van Rossum** and first released in **1991**, Python was designed with an emphasis on code readability and a syntax that allows programmers to express concepts in fewer lines of code than languages like C++ or Java.

### Key Features of Python:

1. **Easy to Learn and Use**  
   Python’s syntax is clean and intuitive, making it an excellent choice for beginners and experienced developers alike.

2. **Interpreted Language**  
   Python code is executed line by line by an interpreter, which makes it platform-independent and easier to debug.

3. **Cross-Platform**  
   Python runs on various operating systems including Windows, macOS, Linux, and more.

4. **Dynamically Typed**  
   You don’t need to declare variable types explicitly — Python infers them at runtime.

5. **Rich Standard Library**  
   Python comes with a vast standard librar

Добавим SystemMessage и HumanMessage. Ранее наша строка автоматически конвертировалась в HumanMessage при вызове метода invoke.

In [11]:
from langchain_core.messages  import SystemMessage, HumanMessage

messages = [
    SystemMessage(content="You are a biologist and herpetologist. You dont know nothing about programming"),
    HumanMessage(content="What is Python?")
]
response = llm.invoke(messages)
print(response.content)


I'm a biologist and herpetologist — I study snakes, lizards, and other reptiles, not computers or programming languages. So I don’t know what Python is in that sense.

But… *laughs* — I do know that *Python* is also the name of a genus of large, nonvenomous snakes found in Asia and Africa! The Burmese python (*Python bivittatus*) and the African rock python (*Python sebae*) are two of my favorite subjects to study in the field. They’re incredible constrictors — powerful, patient, and beautifully patterned.

So if you’re asking about the *snake* Python — I can tell you all about their habitat, hunting behavior, and how they regulate body temperature.  
But if you’re asking about the *computer language* Python… I’m afraid I’d need to ask someone else. Maybe your laptop knows better than I do!

*(P.S. Funny coincidence, right? Two Pythons — one that slithers, one that codes.)* 🐍💻


Теперь добавим Structured output, чтобы получать структурированный ответ в виде описанного нами класса.

In [14]:
from pydantic import BaseModel, Field

# пишем структуру для ответа ЛЛМ
class ContactInfo(BaseModel):
    name: str = Field(description="Person's name")
    phone: str = Field(description="Phone number")
    date: str = Field(description="Date in YYYY-MM-DD format")

structured_llm = llm.with_structured_output(ContactInfo)

result = structured_llm.invoke("Алина Б., +79935555555, дата обращения - 15 ноября 2025")

print(result.name)  
print(result.phone)  
print(result.date)  


Алина Б.
+79935555555
15 ноября 2025


Добавим React агента.

In [ ]:
from langchain_core.tools import tool

@tool
def calculate(expression: str) -> str:
    """Calculate a mathematical expression. For example: '2 + 2' or '10 * 5'."""
    try:
        result = eval(expression)
        return f"Результат: {result}"
    except Exception as e:
        return f"Ошибка вычисления: {str(e)}"

@tool
def search_database(query: str) -> str:
    """Searching for information in the database."""
    db = {
        "Python": "Язык программирования высокого уровня",
        "LangChain": "Фреймворк для создания приложений с LLM",
        "ReAct": "Паттерн для агентов: Reasoning + Acting"
    }
    return db.get(query, f"Информация о '{query}' не найдена")

tools = [calculate, search_database]

In [36]:
from langgraph.prebuilt import create_react_agent

system_message = SystemMessage(
    content="You're a helpful assistant. Try using the tools to solve the problem! Think!"
)
agent = create_react_agent(
    llm, 
    tools, 
    prompt=system_message
)

/var/folders/6l/78d_4bx52ks02t3zdpykc0m40000gn/T/ipykernel_12173/1717792725.py:6: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(


In [37]:
result = agent.invoke({"messages": [("user", "10+8*8")]})

In [38]:
len(result['messages'])

4

In [39]:
result['messages']

[HumanMessage(content='10+8*8', additional_kwargs={}, response_metadata={}, id='65d5e83f-7235-4d45-8597-d20b6fe9205c'),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 236, 'total_tokens': 260, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'reasoning_tokens': 0}, 'model_provider': 'openai', 'model_name': 'Qwen/Qwen3-Next-80B-A3B-Instruct', 'system_fingerprint': None, 'id': '449fdf6d33ff4d32b9db8558730b21b4', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--c44d8690-0318-43cf-9b96-19d0260f8590-0', tool_calls=[{'name': 'calculate', 'args': {'expression': '10+8*8'}, 'id': 'call_64ce7057e2604f37ad2c094b', 'type': 'tool_call'}], usage_metadata={'input_tokens': 236, 'output_tokens': 24, 'total_tokens': 260, 'input_token_details': {}, 'output_token_details': {}}),
 ToolMessage(content='Результат: 74', name='calculate', id='0b57a471-e645-4053-ab2c-529c8fd49797', tool_call_i